# [Deep MNIST for Experts](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html)

## [Load MNIST Data](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#load-mnist-data)

In [1]:
# MNISTはexamplesにあって、ダウンロードもメソッド一発
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# mnist datasetにはtrain, validation, testの３つのデータが入っている
# それぞれDataSetというクラス(mnist自体はDatasetsというクラス)
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x103e7f240>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1041538d0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x10414f2e8>)

In [3]:
# イメージ数は
# (ndarrayの次元はshapeメソッドを使う)
# train: 55000 images
# validation: 5000 images
# validation: 10000 images

# イメージはそれぞれ size 784 のarray. 28 x 28 pixelの画像
print(mnist.train.images.shape)
print(mnist.validation.images.shape)
print(mnist.test.images.shape)

(55000, 784)
(5000, 784)
(10000, 784)


In [4]:
# それぞれのイメージがどの数字を表しているかはlabelsに入っている
# 0~9のどこかに1が入っている
print(mnist.train.labels.shape)
print(mnist.validation.labels.shape)
print(mnist.test.labels.shape)

(55000, 10)
(5000, 10)
(10000, 10)


# [Start TensorFlow InteractiveSession](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#start-tensorflow-interactivesession)

In [5]:
import tensorflow as tf

# Interactive Sessionを使うとGraphを動的に生成できる
# (通常のSessionはsessionを作成する前にGraphを全て生成していないといけない)
sess = tf.InteractiveSession()

# Computation Graph
# Build a Softmax Regression Model
# [Placeholders](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#placeholders)

In [6]:
# xが入力で、y_が正解
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [10]:
# xW + b
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.initialize_all_variables())

# y = softmax(xW + b)
y = tf.nn.softmax(tf.matmul(x,W) + b)

# lossはcross entropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# [Train the Model](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#train-the-model)

In [12]:
# Gradient Descentを使った学習ステップをLearning Rate 0.5で1ステップ実行する計算グラフ
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [13]:
# mini_batchサイズ50で、train_stepを1000ステップ実施
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# [Evaluate the Model](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#evaluate-the-model)

In [14]:
# 正答率を計算
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


# [Building a Multilayer Convolutional Network](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#build-a-multilayer-convolutional-network)

Getting 91% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99.2% accuracy -- not state of the art, but respectable.

## [Weight Initialization](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#weight-initialization)

In [16]:
# To create this model, we're going to need to create a lot of weights and biases. 
# One should generally initialize weights with a small amount of noise for symmetry 
# breaking, and to prevent 0 gradients
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# Since we're using ReLU neurons, it is also good practice to initialize them 
# with a slightly positive initial bias to avoid "dead neurons." Instead of doing 
# this repeatedly while we build the model, let's create two handy functions 
# to do it for us.
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

## [Convolution and Pooling](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#convolution-and-pooling)

TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. 

In [21]:
# https://www.tensorflow.org/versions/master/api_docs/python/nn.html#conv2d
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [22]:
# https://www.tensorflow.org/versions/master/api_docs/python/nn.html#max_pool
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## [First Convolutional Layer](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#first-convolutional-layer)

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

In [23]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

In [24]:
x_image = tf.reshape(x, [-1,28,28,1])

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.

In [25]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## [Second Convolutional Layer](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#second-convolutional-layer)

In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [26]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## [Densely Connected Layer](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#densely-connected-layer)

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [27]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.1

In [28]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## [Readout Layer](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#readout-layer)

Finally, we add a softmax layer, just like for the one layer softmax regression above.

In [29]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## [Train and Evaluate the Model](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html#train-and-evaluate-the-model)

How well does this model do? To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network above. The differences are that: we will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer; we will include the additional parameter keep_prob in feed_dict to control the dropout rate; and we will add logging to every 100th iteration in the training process.

In [30]:
# loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# accuracy
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# train
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# result
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.76
step 200, training accuracy 0.94
step 300, training accuracy 0.8
step 400, training accuracy 0.9
step 500, training accuracy 0.9
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 1
step 900, training accuracy 0.98
step 1000, training accuracy 0.94
step 1100, training accuracy 0.98
step 1200, training accuracy 0.92
step 1300, training accuracy 0.94
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 0.94
step 1700, training accuracy 0.98
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 0.96
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 0.92
step 2700, training accuracy 0.96
step 2800, training accuracy 0.96
step 2900, training accuracy 1
step 3000, tra